# Repressilator

In [ ]:
pip install -e .

In [ ]:
import loica as lc
import matplotlib.pyplot as plt
import networkx as nx
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Mock numbers

In [ ]:
def growth_rate(t):
    return lc.gompertz_growth_rate(t, 0.01, 1, 1, 1)

def biomass(t):
    return lc.gompertz(t, 0.01, 1, 1, 1)

metab = lc.SimulatedMetabolism("Simulated metabolism", biomass, growth_rate)
genetic_n = lc.GeneticNetwork()

In [ ]:
strain = lc.Strain('Strain', genetic_n, metab)

In [ ]:
# Transcription factors
laci = lc.Regulator(name='LacI', degradation_rate=1, init_concentration=10)
tetr = lc.Regulator(name='TetR', degradation_rate=1)
ci = lc.Regulator(name='cI', degradation_rate=1)

genetic_n.add_regulator([tetr, laci, ci])

In [ ]:
mcherry = lc.Reporter(name='mCherry', degradation_rate=1, color='red') 
cyan = lc.Reporter(name='CFP', degradation_rate=1, color='blue') 
mven = lc.Reporter(name='mVenus', degradation_rate=1, color='yellow') 

genetic_n.add_reporter([mcherry, cyan, mven])

In [ ]:
laci_not_tetr = lc.Hill1(name='Plac', input=laci, output=[tetr, mcherry], alpha=[10000, 0], K=1, n=2)
tetr_not_ci = lc.Hill1(name='Ptet', input=tetr, output=[ci, mven], alpha=[10000, 0], K=1, n=2)
ci_not_laci = lc.Hill1(name='PcI', input=ci, output=[laci, cyan], alpha=[10000, 0], K=1, n=2)

genetic_n.add_operator([laci_not_tetr, tetr_not_ci, ci_not_laci])

In [ ]:
plt.figure(figsize=(3.5,3.5), dpi=100)
genetic_n.draw()

In [ ]:
sample = lc.Sample(strain=strain)
assay = lc.Assay([sample], 
                n_measurements=100, 
                interval=0.24,
                name='Extended repressilator',
                description='Simulated expression of extended repressilator strain'
                )

In [ ]:
assay.run(substeps=10, mode='track_all')

In [ ]:
m = assay.measurements
fig,ax = plt.subplots()
m[m.Signal=='mCherry in Strain'].plot(x='Time', y='Measurement', color='red', style='-', ax=ax)
m[m.Signal=='CFP in Strain'].plot(x='Time', y='Measurement', color='cyan', style='-', ax=ax)
m[m.Signal=='mVenus in Strain'].plot(x='Time', y='Measurement', color='yellow', style='-', ax=ax)
ax.set_xlabel('Time (h)')
ax.set_ylabel('Concentration (M)')

fig1,ax1 = plt.subplots()
m[m.Signal=='LacI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax1)
m[m.Signal=='TetR in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax1)
m[m.Signal=='cI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax1)
ax1.set_xlabel('Time (h)')
ax1.set_ylabel('Concentration (M)')

fig2, ax2 = plt.subplots()
m[m.Signal=='Strain biomass'].plot(x='Time', y='Measurement', style='-', ax=ax2)
ax2.set_xlabel('Time (h)')
ax2.set_ylabel('OD 600')

ax.legend(['mCherry', 'CFP', 'mVenus'])
ax1.legend(['LacI', 'TetR', 'cI'])
ax2.legend(['Biomass'])